In [1]:
pip install imbalanced-learn

     |████████████████████████████████| 167 kB 4.0 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_7196538281784381bd575ee5aa302a45.csv"
df=pd.read_csv(url)
df.head()

,name,mcg,gvh,alm,mit,erl,pox,vac,nuc,class
0,ADT1_YEAST,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,MIT
1,ADT2_YEAST,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,MIT
2,ADT3_YEAST,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,MIT
3,AAR2_YEAST,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22,NUC
4,AATM_YEAST,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,MIT


In [2]:
df=df.drop(["name"],axis=1)
df.shape

(1484, 9)

In [3]:
import numpy as np
y=df[["class"]]
y.groupby("class")["class"].count()

class
CYT    463
ERL      5
EXC     35
ME1     44
ME2     51
ME3    163
MIT    244
NUC    429
POX     20
VAC     30
Name: class, dtype: int64

In [4]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y=pd.DataFrame(le.fit_transform(y),columns=y.columns)
y.head()

/srv/conda/envs/notebook/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


,class
0,6
1,6
2,6
3,7
4,6


In [5]:
y.groupby("class")["class"].count()/len(y)

class
0    0.311995
1    0.003369
2    0.023585
3    0.029650
4    0.034367
5    0.109838
6    0.164420
7    0.289084
8    0.013477
9    0.020216
Name: class, dtype: float64

In [6]:
X=df.drop(["class"],axis=1)
X.head()

,mcg,gvh,alm,mit,erl,pox,vac,nuc
0,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22
1,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22
2,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22
3,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22
4,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22


In [7]:
import numpy as np
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

# feature extraction. We are going to output the selection scores for all features and select the features with the highest scores.
test = SelectKBest(score_func=f_classif, k="all")
fit = test.fit(X, np.asarray(y).reshape(y.shape[0],))
features = fit.transform(X)
# summarize scores
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
2,138.798404
5,116.074130
0,100.729393
4,90.884060
1,72.984406
3,51.692897
7,22.013807
6,4.878718


In [8]:
X_scores=scores[scores["scores"]>0]
len(X_scores)

8

In [9]:
X_prime=X.iloc[:,X_scores.index]
X_prime.head()

,alm,pox,mcg,erl,gvh,mit,nuc,vac
0,0.47,0.0,0.58,0.5,0.61,0.13,0.22,0.48
1,0.48,0.0,0.43,0.5,0.67,0.27,0.22,0.53
2,0.49,0.0,0.64,0.5,0.62,0.15,0.22,0.53
3,0.57,0.0,0.58,0.5,0.44,0.13,0.22,0.54
4,0.48,0.0,0.42,0.5,0.44,0.54,0.22,0.48


In [10]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_prime=pd.DataFrame(scaler.fit_transform(X_prime),columns=X_prime.columns)
X_prime.head()

,alm,pox,mcg,erl,gvh,mit,nuc,vac
0,-0.346645,-0.099131,0.581981,-0.09759,0.888481,-0.957203,-0.527919,-0.344175
1,-0.231226,-0.099131,-0.510891,-0.09759,1.372811,0.064312,-0.527919,0.521219
2,-0.115808,-0.099131,1.019130,-0.09759,0.969203,-0.811272,-0.527919,0.521219
3,0.807542,-0.099131,0.581981,-0.09759,-0.483786,-0.957203,-0.527919,0.694298
4,-0.231226,-0.099131,-0.583749,-0.09759,-0.483786,2.034375,-0.527919,-0.344175


In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_prime,y,test_size=0.202,random_state=42)
print(X_train.shape)
print(X_test.shape)

(1184, 8)
(300, 8)


In [12]:
train=pd.concat([X_train,y_train],axis=1)

train0=train[train["class"]==0]
train1=train[train["class"]==1]
train2=train[train["class"]==2]
train3=train[train["class"]==3]
train4=train[train["class"]==4]
train5=train[train["class"]==5]
train6=train[train["class"]==6]
train7=train[train["class"]==7]
train8=train[train["class"]==8]
train9=train[train["class"]==9]

#train1=pd.concat([train1]*3,axis=0)
minority=pd.concat([train1,train2,train3,train4,train8,train9],axis=0)
minority=pd.concat([minority]*2,axis=0)

train=pd.concat([train0,minority,train5,train6,train7],axis=0)

X_train=train.drop(["class"],axis=1)
y_train=train[["class"]]

print(X_train.shape)

(1339, 8)


In [13]:
from sklearn.ensemble import RandomForestClassifier
Forest=RandomForestClassifier(random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
Forest

RandomForestClassifier(random_state=0)

In [14]:
from sklearn.metrics import accuracy_score
train_pred=Forest.predict(X_train)
test_pred=Forest.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  1.0
Accuracy on test data:  0.61


In [15]:
Forest=RandomForestClassifier(n_estimators=99,n_jobs=-1,ccp_alpha=0.002,random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
Forest

RandomForestClassifier(ccp_alpha=0.002, n_estimators=99, n_jobs=-1,
                       random_state=0)

In [16]:
train_pred=Forest.predict(X_train)
test_pred=Forest.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.8200149365197908
Accuracy on test data:  0.62


In [17]:
from sklearn.metrics import f1_score
print("F1 score on train data: ",f1_score(y_train,train_pred,average="micro"))
print("F1 score on test data: ",f1_score(y_test,test_pred,average="micro"))

F1 score on train data:  0.8200149365197908
F1 score on test data:  0.62


In [18]:
from sklearn.metrics import precision_score
print("Precision score on train data: ",precision_score(y_train,train_pred,average="micro"))
print("Precision score on test data: ",precision_score(y_test,test_pred,average="micro"))

Precision score on train data:  0.8200149365197908
Precision score on test data:  0.62


In [19]:
from sklearn.metrics import recall_score
print("Recall score on train data: ",recall_score(y_train,train_pred,average="micro"))
print("Recall score on test data: ",recall_score(y_test,test_pred,average="micro"))

Recall score on train data:  0.8200149365197908
Recall score on test data:  0.62


In [20]:

from sklearn.metrics import confusion_matrix
print("Confusion matrix on train data: \n",confusion_matrix(y_train,train_pred))
print("Confusion matrix on test data: \n",confusion_matrix(y_test,test_pred))

Confusion matrix on train data: 
 [[305   0   0   0   1   4  11  42   1   0]
 [  0  10   0   0   0   0   0   0   0   0]
 [  2   0  50   0   2   0   0   0   0   0]
 [  0   0   2  70   0   0   0   0   0   0]
 [  0   0   0   2  80   6   0   2   0   0]
 [  5   0   0   0   1 116   1   6   0   0]
 [ 36   0   0   1   3   4 140   8   0   0]
 [ 47   0   0   0   1   9  14 273   0   0]
 [  6   0   0   0   0   0   2   0  28   0]
 [ 12   0   0   0   2   6   0   2   0  26]]
Confusion matrix on test data: 
 [[63  1  0  0  3  3 29  0  0]
 [ 2  4  0  0  0  2  0  0  0]
 [ 0  3  5  0  0  0  0  0  0]
 [ 1  0  1  3  0  1  0  0  0]
 [ 0  0  0  0 31  1  2  0  0]
 [14  0  0  2  3 27  6  0  0]
 [23  0  0  1  7  3 51  0  0]
 [ 0  0  0  0  0  0  0  2  0]
 [ 2  2  0  0  0  1  1  0  0]]


The above results used standardized data. 

In [21]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.202,random_state=42)
print(X_train.shape)
print(X_test.shape)

(1184, 8)
(300, 8)


In [22]:
train=pd.concat([X_train,y_train],axis=1)

train0=train[train["class"]==0]
train1=train[train["class"]==1]
train2=train[train["class"]==2]
train3=train[train["class"]==3]
train4=train[train["class"]==4]
train5=train[train["class"]==5]
train6=train[train["class"]==6]
train7=train[train["class"]==7]
train8=train[train["class"]==8]
train9=train[train["class"]==9]

#train1=pd.concat([train1]*3,axis=0)
minority=pd.concat([train1,train2,train3,train4,train8,train9],axis=0)
minority=pd.concat([minority]*2,axis=0)

train=pd.concat([train0,minority,train5,train6,train7],axis=0)

X_train=train.drop(["class"],axis=1)
y_train=train[["class"]]

print(X_train.shape)

(1339, 8)


In [23]:
from sklearn.ensemble import RandomForestClassifier
Forest=RandomForestClassifier(random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
Forest

RandomForestClassifier(random_state=0)

In [24]:
from sklearn.metrics import accuracy_score
train_pred=Forest.predict(X_train)
test_pred=Forest.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  1.0
Accuracy on test data:  0.62


In [25]:
Forest=RandomForestClassifier(n_estimators=99,n_jobs=-1,ccp_alpha=0.002,random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
Forest

RandomForestClassifier(ccp_alpha=0.002, n_estimators=99, n_jobs=-1,
                       random_state=0)

In [26]:
train_pred=Forest.predict(X_train)
test_pred=Forest.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.8267363704256908
Accuracy on test data:  0.63


With the same hyper-parameter values, using raw data increased the model's accuracy by 1%. From this point on, I will use raw data.

In [27]:
from sklearn.ensemble import GradientBoostingClassifier
Gradient=GradientBoostingClassifier(random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
Gradient

GradientBoostingClassifier(random_state=0)

In [28]:
train_pred=Gradient.predict(X_train)
test_pred=Gradient.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.8678117998506348
Accuracy on test data:  0.5566666666666666


In [29]:
Gradient=GradientBoostingClassifier(learning_rate=0.015,subsample=0.5,criterion="mse",
                                    random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
Gradient

GradientBoostingClassifier(criterion='mse', learning_rate=0.015, random_state=0,
                           subsample=0.5)

In [30]:
train_pred=Gradient.predict(X_train)
test_pred=Gradient.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.6885735623599701
Accuracy on test data:  0.62


In [31]:
print("F1 score on train data: ",f1_score(y_train,train_pred,average="micro"))
print("F1 score on test data: ",f1_score(y_test,test_pred,average="micro"))

F1 score on train data:  0.6885735623599701
F1 score on test data:  0.62


In [32]:
from sklearn import svm
SVM=svm.SVC(random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
SVM

SVC(random_state=0)

In [33]:
train_pred=SVM.predict(X_train)
test_pred=SVM.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.619865571321882
Accuracy on test data:  0.6166666666666667


In [34]:
SVM=svm.SVC(C=0.5,random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
SVM

SVC(C=0.5, random_state=0)

In [35]:
train_pred=SVM.predict(X_train)
test_pred=SVM.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.6094100074682599
Accuracy on test data:  0.62


In [36]:
print("F1 score on train data: ",f1_score(y_train,train_pred,average="micro"))
print("F1 score on test data: ",f1_score(y_test,test_pred,average="micro"))

F1 score on train data:  0.6094100074682599
F1 score on test data:  0.62


In [37]:
from imblearn.ensemble import BalancedRandomForestClassifier
BForest=BalancedRandomForestClassifier(
                                       random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
BForest

BalancedRandomForestClassifier(random_state=0)

In [38]:
train_pred=BForest.predict(X_train)
test_pred=BForest.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.6415235250186706
Accuracy on test data:  0.5266666666666666


In [39]:
BForest=BalancedRandomForestClassifier(n_estimators=110,min_samples_split=3,min_samples_leaf=4,
                                       random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
BForest

BalancedRandomForestClassifier(min_samples_leaf=4, min_samples_split=3,
                               n_estimators=110, random_state=0)

In [40]:
train_pred=BForest.predict(X_train)
test_pred=BForest.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.5608663181478716
Accuracy on test data:  0.5766666666666667


In [41]:
print("F1 score on train data: ",f1_score(y_train,train_pred,average="micro"))
print("F1 score on test data: ",f1_score(y_test,test_pred,average="micro"))

F1 score on train data:  0.5608663181478716
F1 score on test data:  0.5766666666666667


How would the model have performed if minority classes were not duplicated in traind ata?

In [42]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.202,random_state=42)
print(X_train.shape)
print(X_test.shape)

(1184, 8)
(300, 8)


In [43]:
Forest=RandomForestClassifier(random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
Forest

RandomForestClassifier(random_state=0)

In [44]:
train_pred=Forest.predict(X_train)
test_pred=Forest.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  1.0
Accuracy on test data:  0.59


In [45]:
Forest=RandomForestClassifier(n_estimators=99,n_jobs=-1,ccp_alpha=0.002,random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
Forest

RandomForestClassifier(ccp_alpha=0.002, n_estimators=99, n_jobs=-1,
                       random_state=0)

In [46]:
train_pred=Forest.predict(X_train)
test_pred=Forest.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.8175675675675675
Accuracy on test data:  0.6266666666666667


In [47]:
print("F1 score on train data: ",f1_score(y_train,train_pred,average="micro"))
print("F1 score on test data: ",f1_score(y_test,test_pred,average="micro"))

F1 score on train data:  0.8175675675675675
F1 score on test data:  0.6266666666666667


In [48]:
Gradient=GradientBoostingClassifier(random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
Gradient

GradientBoostingClassifier(random_state=0)

In [49]:
train_pred=Gradient.predict(X_train)
test_pred=Gradient.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.8547297297297297
Accuracy on test data:  0.5833333333333334


In [50]:
Gradient=GradientBoostingClassifier(learning_rate=0.015,subsample=0.5,criterion="mse",
                                    random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
Gradient

GradientBoostingClassifier(criterion='mse', learning_rate=0.015, random_state=0,
                           subsample=0.5)

In [51]:
train_pred=Gradient.predict(X_train)
test_pred=Gradient.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.6883445945945946
Accuracy on test data:  0.62


In [52]:
print("F1 score on train data: ",f1_score(y_train,train_pred,average="micro"))
print("F1 score on test data: ",f1_score(y_test,test_pred,average="micro"))

F1 score on train data:  0.6883445945945946
F1 score on test data:  0.62


In [53]:
SVM=svm.SVC(random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
SVM

SVC(random_state=0)

In [54]:
train_pred=SVM.predict(X_train)
test_pred=SVM.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.6275337837837838
Accuracy on test data:  0.6133333333333333


In [55]:
SVM=svm.SVC(C=0.5,random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
SVM

SVC(C=0.5, random_state=0)

In [56]:
train_pred=SVM.predict(X_train)
test_pred=SVM.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.6114864864864865
Accuracy on test data:  0.6266666666666667


In [57]:
print("F1 score on train data: ",f1_score(y_train,train_pred,average="micro"))
print("F1 score on test data: ",f1_score(y_test,test_pred,average="micro"))

F1 score on train data:  0.6114864864864865
F1 score on test data:  0.6266666666666667


In [58]:
BForest=BalancedRandomForestClassifier(
                                       random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
BForest

BalancedRandomForestClassifier(random_state=0)

In [59]:
train_pred=BForest.predict(X_train)
test_pred=BForest.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.5244932432432432
Accuracy on test data:  0.4666666666666667


In [78]:
BForest=BalancedRandomForestClassifier(n_estimators=90,min_samples_split=3,min_samples_leaf=4,
                                       random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
BForest

BalancedRandomForestClassifier(min_samples_leaf=4, min_samples_split=3,
                               n_estimators=90, random_state=0)

In [79]:
train_pred=BForest.predict(X_train)
test_pred=BForest.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.5008445945945946
Accuracy on test data:  0.48333333333333334


In [73]:
print("F1 score on train data: ",f1_score(y_train,train_pred,average="micro"))
print("F1 score on test data: ",f1_score(y_test,test_pred,average="micro"))

F1 score on train data:  0.5008445945945946
F1 score on test data:  0.48333333333333334


## Summary

#### Except the first experimental model, all models were built using raw data, since the experimental model showed that using raw data resulted in slightly higher accuracy.

#### With no data resampling and default values for hyper-parameters

|Model|Train data accuracy|Test data accuracy|
|-----|-------------------|------------------|
|RandomForestClassifier|100%|59%|
|GradientBoostingClassifier|85.47%|58.33%|
|Support Vector Machine|62.15%|61.33%|
|BalancedRandomForestClassifier|52.45%|46.67%|

#### With no data resampling but tuned values for the impactful hyer-parameters.

|Model|Train data accuracy|Train data F1 score|Test data accuracy|Test data F1 score|
|-----|-------------------|------------------|-------------------|------------------|
|RandomForestClassifier|81.76%|81.76%|62.76%|62.76%|
|GradientBoostingClassifier|68.83%|68.83%|62%|62%|
|Support Vector Machine (SVM)|61.15%|61.15%|62.67%|62.67%|
|BalancedRandomForestClassifier|50.08%|50.08%|48.33%|48.33%|

#### Fine-tuning the impactful hyper-parameters reduced the models' accuracies on train data but improved their accuracies on test data. Hyper-parameter fine-tuning, in this case, reduced overfitting. A model's robustness is often measured on test data, not train data.

#### With the minority classes in train data duplicated by 2 times and default values for hyper-paramters

|Model|Train data accuracy|Test data accuracy|
|-----|-------------------|------------------|
|RandomForestClassifier|100%| 62%|
|GradientBoostingClassifier|86.78%|55.67%|
|Support Vector Machine (SVM)|61.99%|61.67%|
|BalancedRandomForestClassifier|64.15%|52.67%|

#### With minority classes in train data duplicated by 2 times and the impactful hyper-parameters tuned.

|Model|Train data accuracy|Train data F1 score|Test data accuracy|Test data F1 score|
|-----|-------------------|-------------------|------------------|------------------|
|RandomForestClassifier|82.67%|82.67%|63%|63%|
|GradientBoostingClassifier|68.86%|68.86%|62%|62%|
|Support Vector Machine (SVM)|60.94%|60.94%|62%|62%|
|BalancedRandomForestClassifier|56.09%|56.09%|57.67%|57.67%|

1. When exposure to minority classes is increased by duplicating the minority classes by 2 times in train data, except the GradientBoostingClassifier model, all models delivered higher accuracies on test data.
2. The best model is the RandomForestClassifier model when exposure to minority classes is increased by duplicating those classes by 2 times in train data and with the impactful hyper-parameters tuned.The model had an accuracy and F1 score of 63% on test data.
3. Increasing the model's exposure to minority classes by duplicating those classes in train data does not guarantee better performance in test data. **The impact depends on the dataset in hand and is therefore case-specific.** Nonetheless, it is a quick and effortless way to tackle imbalanced class distribution. For this dataset, the strategy did improve the best model's performance on test data by 1%. Note that even BalancedRandomForestClassifier model, which has built-in functions to handle class imbalance, benefited from the increased exposure to minority classes in train data. 